<a href="https://colab.research.google.com/github/DongyangLi12/sentiment-classification/blob/main/%E2%80%9CFull_NER_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install scikit-learn


In [ ]:
from collections import defaultdict
from urllib import request
import json
import pandas as pd
from sklearn.metrics import f1_score
from math import ceil
from transformers import AutoModel, AutoTokenizer
from tqdm.auto import tqdm

In [ ]:
import random
from random import shuffle
import numpy as np
import torch
import torch.nn as nn

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
def parse_conllu_using_pandas(block):
    records = []
    for line in block.splitlines():
        if not line.startswith('#'):
            records.append(line.strip().split('\t'))
    return pd.DataFrame.from_records(
        records,
        columns=['ID', 'FORM', 'TAG', 'Misc1', 'Misc2'])

In [ ]:
def tokens_to_labels(df):
    return (
        df.FORM.tolist(),
        df.TAG.tolist()
    )

In [ ]:
PREFIX = "https://raw.githubusercontent.com/UniversalNER/"
DATA_URLS = {
    "en_ewt": {
        "train": "UNER_English-EWT/master/en_ewt-ud-train.iob2",
        "dev": "UNER_English-EWT/master/en_ewt-ud-dev.iob2",
        "test": "UNER_English-EWT/master/en_ewt-ud-test.iob2"
    },
    "en_pud": {
        "test": "UNER_English-PUD/master/en_pud-ud-test.iob2"
    }
}

In [ ]:
# en_ewt is the main train-dev-test split
# en_pud is the OOD test set
data_dict = defaultdict(dict)
for corpus, split_dict in DATA_URLS.items():
    for split, url_suffix in split_dict.items():
        url = PREFIX + url_suffix
        with request.urlopen(url) as response:
            txt = response.read().decode('utf-8')
            data_frames = map(parse_conllu_using_pandas,
                              txt.strip().split('\n\n'))
            token_label_alignments = list(map(tokens_to_labels,
                                              data_frames))
            data_dict[corpus][split] = token_label_alignments

In [ ]:
# Saving the data so that you don't have to redownload it each time.
with open('ner_data_dict.json', 'w', encoding='utf-8') as out:
    json.dump(data_dict, out, indent=2, ensure_ascii=False)

In [ ]:
with open('ner_data_dict.json', 'r', encoding='utf-8') as f:
    data_dict = json.load(f)

Organize the input as list of 'word-tag' pairs

In [ ]:
train_set = [[[word, label] for word, label in zip(words, labels)] for words, labels in data_dict['en_ewt']['train']]
dev_set = [[[word, label] for word, label in zip(words, labels)] for words, labels in data_dict['en_ewt']['dev']]
test_set = [[[word, label] for word, label in zip(words, labels)] for words, labels in data_dict['en_ewt']['test']]
test_set_ood =  [[[word, label] for word, label in zip(words, labels)] for words, labels in data_dict['en_pud']['test']]

Take a look at the original labels

In [ ]:
labels = set()
for ex in train_set:

    labels.update([el[1] for el in ex])
n_classes = len(labels)
sorted(labels)

['B-LOC', 'B-ORG', 'B-PER', 'I-LOC', 'I-ORG', 'I-PER', 'O']

In [ ]:
label_to_i = {
    label: i
    for i, label in enumerate(sorted(labels))
}
i_to_label = {
    i: label
    for label, i in label_to_i.items()
}

In [ ]:
label_to_i

{'B-LOC': 0,
 'B-ORG': 1,
 'B-PER': 2,
 'I-LOC': 3,
 'I-ORG': 4,
 'I-PER': 5,
 'O': 6}

Using Bert to do the original tag NER task

In [ ]:
model_tag = 'google-bert/bert-base-uncased'

tokeniser = AutoTokenizer.from_pretrained(model_tag)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
class ClassificationHead(nn.Module):
    def __init__(self, model_dim=768, n_classes=7, dropout_prob=0.1):
        super().__init__()
        self.dropout = nn.Dropout(dropout_prob)
        self.linear  = nn.Linear(model_dim, n_classes)

    def forward(self, x):

        x = self.dropout(x)
        return self.linear(x)

In [ ]:
def process_sentence_batch(batch_inputs, label_to_i,
                           tokeniser, encoder, clf_head,
                           encoder_device, clf_head_device):

    # process all sentences in the batch
    gold_labels = []
    words_list = []
    all_first_subword_embeddings = []


    for sentence in batch_inputs:
        sentence_labels = [label_to_i[label] for _, label in sentence]
        gold_labels.append(sentence_labels)

        words = [word for word, _ in sentence]
        words_list.append(words)


    tokenization = tokeniser(
        words_list,
        is_split_into_words=True,
        padding='longest',
        truncation=False,
        return_tensors='pt'
    )


    inputs = {k: v.to(encoder_device) for k, v in tokenization.items()}


    # delete [CLS] and [SEP]
    outputs = encoder(**inputs).last_hidden_state[:, 1:-1, :]

    # extract the first subword embedding for each word
    for i in range(len(batch_inputs)):
        word_ids = tokenization.word_ids(batch_index=i)
        seen = set()
        first_sub_embs = []
        for j, wid in enumerate(word_ids[1:-1]):
            if wid is None or wid in seen:
                continue
            first_sub_embs.append(outputs[i, j])
            seen.add(wid)

        sent_emb = torch.stack(first_sub_embs)
        all_first_subword_embeddings.append(sent_emb)


    gold_label_tensors = [
        torch.tensor(lbls, dtype=torch.long).to(clf_head_device)
        for lbls in gold_labels
    ]

    # feed to the classification head sentence by sentence
    logits_list = []
    for sent_emb in all_first_subword_embeddings:
        # sent_emb: (num_words, hidden_size)
        logits = clf_head(sent_emb.to(clf_head_device))
        # logits: (num_words, num_classes)
        logits_list.append(logits)

    return logits_list, gold_label_tensors


In [ ]:
def train_epoch(train_data, label_to_i, batch_size, tokeniser, encoder, clf_head,
                encoder_device, clf_head_device, loss_fn, optimiser):
    encoder.train()
    n_steps = ceil(len(train_data) / batch_size)
    if len(train_data) - len(train_data) // batch_size * batch_size:
        n_steps -= 1
    epoch_losses = torch.empty(n_steps)
    for step_n in tqdm(range(n_steps), leave=False, desc='Train'):
       lo = step_n * batch_size
       hi = lo + batch_size
       batch_texts = train_data[lo:hi]

       logits_list, gold_label_tensors = process_sentence_batch(batch_texts, label_to_i,
                           tokeniser, encoder, clf_head,
                           encoder_device, clf_head_device)

       # concatenate the logits of all the sentences
       batch_logits = torch.cat(logits_list, dim=0)
       # concatenate all the labels
       batch_labels = torch.cat(gold_label_tensors, dim=0)

       loss = loss_fn(batch_logits, batch_labels)
       loss.backward()
       optimiser.step()
       optimiser.zero_grad()
       epoch_losses[step_n] = loss.item()
    return epoch_losses.mean().item()

In [ ]:

def get_spans(label_ids, i_to_label):
    labeled = []
    unlabeled = []
    start = None
    cur_label = None

    for i, lid in enumerate(label_ids):
        tag = i_to_label[lid]

        if tag.startswith('B-'):
          if start is not None:
            labeled.append((start, i-1, cur_label))

          start = i
          cur_label = tag[2:]

        elif tag.startswith('I-') and start is not None and tag[2:] == cur_label:
          pass

        else:
            if start is not None:
                labeled.append((start, i-1, cur_label))
            start = None
            cur_label = None


    if start is not None:
        labeled.append((start, len(label_ids)-1, cur_label))

    i = 0
    N = len(label_ids)
    while i < N:
        tag = i_to_label[label_ids[i]]
        if tag.startswith('B-'):
            start = i
            j = i + 1
            while j < N and i_to_label[label_ids[j]].startswith('I-'):
                j += 1

            unlabeled.append((start, j-1))
            i = j
        else:
            i += 1

    return labeled, unlabeled

In [ ]:
label_to_i

{'B-LOC': 0,
 'B-ORG': 1,
 'B-PER': 2,
 'I-LOC': 3,
 'I-ORG': 4,
 'I-PER': 5,
 'O': 6}

In [ ]:
def validate_epoch_batch(dev_inputs, label_to_i, tokeniser, encoder, clf_head,
                          encoder_device, clf_head_device, batch_size, i_to_label):
    encoder.eval()

    all_true_labels = []
    all_pred_labels = []

    total_true_lab_spans = 0
    total_true_unlab_spans = 0
    total_pred_lab_spans = 0
    total_pred_unlab_spans = 0
    correct_unlab = 0
    correct_lab = 0


    n_steps = ceil(len(dev_inputs) / batch_size)
    for step_n in tqdm(range(n_steps), leave=False, desc='Eval'):
        lo = step_n * batch_size
        hi = lo + batch_size
        batch_data = dev_inputs[lo:hi]

        with torch.no_grad():
            logits_list, gold_list = process_sentence_batch(
                batch_data, label_to_i,
                tokeniser, encoder, clf_head,
                encoder_device, clf_head_device)



        for logits, gold in zip(logits_list, gold_list):
            pred = torch.argmax(logits, dim=-1).cpu().numpy().tolist()
            gold = gold.cpu().numpy().tolist()

            all_pred_labels.extend(pred)
            all_true_labels.extend(gold)


            true_lab_spans, true_unlab_spans = get_spans(gold, i_to_label)
            pred_lab_spans, pred_unlab_spans = get_spans(pred, i_to_label)

            total_true_lab_spans += len(true_lab_spans)
            total_true_unlab_spans += len(true_unlab_spans)
            total_pred_lab_spans += len(pred_lab_spans)
            total_pred_unlab_spans += len(pred_unlab_spans)

            # unlabeled matching
            correct_unlab += len(set(true_unlab_spans) & set(pred_unlab_spans))
            # Labeled matching
            correct_lab   += len(set(true_lab_spans)   & set(pred_lab_spans))


    # 1) Span‐level matching scores
    unlabeled_span_score = correct_unlab / total_true_unlab_spans if total_true_unlab_spans else 0.0
    labeled_span_score   = correct_lab   / total_true_lab_spans if total_true_lab_spans else 0.0

    # 2) Span‐level labeled Precision/Recall/F1
    precision_span = correct_lab / total_pred_lab_spans if total_pred_lab_spans else 0.0
    recall_span    = correct_lab / total_true_lab_spans if total_true_lab_spans else 0.0
    f1_span = (2 * precision_span * recall_span /
               (precision_span + recall_span)
               if (precision_span + recall_span) else 0.0)

    # 3) Token‐level F1 for 7 tags
    tags_7 = ['B-PER','I-PER','B-LOC','I-LOC','B-ORG','I-ORG','O']
    # map them to indices
    tag_indices = [label_to_i[t] for t in tags_7]
    f1_per_tag = f1_score(all_true_labels, all_pred_labels,
                          labels=tag_indices, average=None)
    token_f1_per_tag = dict(zip(tags_7, f1_per_tag))
    # macro-average over these 7
    macro_token_f1 = f1_score(all_true_labels, all_pred_labels,
                              labels=tag_indices, average='macro')

    return unlabeled_span_score, labeled_span_score, f1_span, token_f1_per_tag, macro_token_f1

In [ ]:
encoder_device = 0  # Can also be 'cpu'
encoder = AutoModel.from_pretrained(
    model_tag).to(encoder_device)
# NB: pass the number of different POS tags
clf_head = ClassificationHead(n_classes=n_classes)
clf_head_device = 0
clf_head.to(clf_head_device);

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
batch_size = 16
n_epochs = 8
patience = 2
best_f1 = 0.0
epochs_no_improve = 0

loss_fn = nn.CrossEntropyLoss()
optimiser = torch.optim.AdamW(
    list(encoder.parameters()) + list(clf_head.parameters()),
    lr=1e-5
)

for epoch_n in range(1, n_epochs+1):
    # Shuffle training set in each epoch
    shuffle(train_set)

    loss = train_epoch(
        train_set, label_to_i, batch_size,
        tokeniser, encoder, clf_head,
        encoder_device, clf_head_device,
        loss_fn, optimiser
    )
    print(f'Epoch {epoch_n} train loss: {loss:.4f}')


    unlabeled_span_score, labeled_span_score, f1_span, token_f1_per_tag, macro_token_f1 = validate_epoch_batch(dev_set, label_to_i,
                                                                                                               tokeniser, encoder, clf_head,
                                                                                                               encoder_device, clf_head_device,
                                                                                                               batch_size, i_to_label)


    print(f"Epoch {epoch_n} Unlabeled span score: {unlabeled_span_score:.4f}")
    print(f"Epoch {epoch_n} Labeled   span score: {labeled_span_score:.4f}")
    print(f"Epoch {epoch_n} Labeled Span F1:      {f1_span:.4f}")

    print("Epoch %d token-level F1 per tag:" % epoch_n)
    for t, f1v in token_f1_per_tag.items():
        print(f"  {t}: {f1v:.4f}")
    print(f"Epoch {epoch_n} token-level Macro F1 (7 tags): {macro_token_f1:.4f}")


    Early Stopping check
    if f1_span > best_f1:
        best_f1 = f1_span
        epochs_no_improve = 0

        torch.save({
            'encoder': encoder.state_dict(),
            'clf_head': clf_head.state_dict(),
            'optim': optimiser.state_dict(),
        }, 'best_model.pt')
        print(f'→ New best F1: {best_f1:.4f}, model saved.')
    else:
        epochs_no_improve += 1
        print(f'→ No improvement for {epochs_no_improve} epoch(s).')
        if epochs_no_improve >= patience:
            print(f'→ Early stopping triggered. Stopping at epoch {epoch_n}.')
            break


Train:   0%|          | 0/783 [00:00<?, ?it/s]

Epoch 1 train loss: 0.0452


Eval:   0%|          | 0/126 [00:00<?, ?it/s]

Epoch 1 Unlabeled span score: 0.8416
Epoch 1 Labeled   span score: 0.8002
Epoch 1 Labeled Span F1:      0.7945
Epoch 1 token-level F1 per tag:
  B-PER: 0.8990
  I-PER: 0.9288
  B-LOC: 0.8452
  I-LOC: 0.7134
  B-ORG: 0.6700
  I-ORG: 0.6361
  O: 0.9933
Epoch 1 token-level Macro F1 (7 tags): 0.8122
→ New best F1: 0.7945, model saved.


Train:   0%|          | 0/783 [00:00<?, ?it/s]

...

Epoch 3 Unlabeled span score: 0.8986
Epoch 3 Labeled   span score: 0.8571
Epoch 3 Labeled Span F1:      0.8019
Epoch 3 token-level F1 per tag:
  B-PER: 0.8942
  I-PER: 0.9641
  B-LOC: 0.8522
  I-LOC: 0.6687
  B-ORG: 0.7103
  I-ORG: 0.7427
  O: 0.9929
Epoch 3 token-level Macro F1 (7 tags): 0.8321
→ No improvement for 1 epoch(s).


Train:   0%|          | 0/783 [00:00<?, ?it/s]

Epoch 4 train loss: 0.0088


Eval:   0%|          | 0/126 [00:00<?, ?it/s]

Epoch 4 Unlabeled span score: 0.8385
Epoch 4 Labeled   span score: 0.7940
Epoch 4 Labeled Span F1:      0.8057
Epoch 4 token-level F1 per tag:
  B-PER: 0.9031
  I-PER: 0.9620
  B-LOC: 0.8575
  I-LOC: 0.7181
  B-ORG: 0.6443
  I-ORG: 0.7062
  O: 0.9937
Epoch 4 token-level Macro F1 (7 tags): 0.8264
→ No improvement for 2 epoch(s).
→ Early stopping triggered. Stopping at epoch 4.


In [ ]:
checkpoint = torch.load('best_model.pt', map_location=lambda s, l: s)  # map_location 根据你的环境调整
encoder.load_state_dict(checkpoint['encoder'])
clf_head.load_state_dict(checkpoint['clf_head'])
optimiser.load_state_dict(checkpoint['optim'])


encoder.eval()
clf_head.eval()

# 2) 定义一个小函数帮忙复用验证逻辑
def test_on_split(split_name, split_data):
    (unlab, lab, f1_span,
     token_f1_per_tag,
     macro_token_f1) = validate_epoch_batch(
        dev_inputs         = split_data,
        label_to_i         = label_to_i,
        tokeniser          = tokeniser,
        encoder            = encoder,
        clf_head           = clf_head,
        encoder_device     = encoder_device,
        clf_head_device    = clf_head_device,
        batch_size         = batch_size,
        i_to_label         = i_to_label
    )
    print(f"\n=== {split_name} ===")
    print(f"Unlabeled span score: {unlab:.4f}")
    print(f"Labeled   span score: {lab:.4f}")
    print(f"Span‐level F1:        {f1_span:.4f}")
    print("Token‐level F1 per tag:")
    for tag, v in token_f1_per_tag.items():
        print(f"  {tag}: {v:.4f}")
    print(f"Token‐level Macro F1: {macro_token_f1:.4f}")

# 3) 分别在两个测试集上跑
test_on_split("In‐Domain Test Set",    test_set)
test_on_split("Out‐of‐Domain Test Set", test_set_ood)

Eval:   0%|          | 0/130 [00:00<?, ?it/s]


=== In‐Domain Test Set ===
Unlabeled span score: 0.8539
Labeled   span score: 0.8217
Span‐level F1:        0.8161
Token‐level F1 per tag:
  B-PER: 0.9341
  I-PER: 0.9281
  B-LOC: 0.8688
  I-LOC: 0.6102
  B-ORG: 0.7239
  I-ORG: 0.7117
  O: 0.9926
Token‐level Macro F1: 0.8242


Eval:   0%|          | 0/63 [00:00<?, ?it/s]


=== Out‐of‐Domain Test Set ===
Unlabeled span score: 0.8316
Labeled   span score: 0.7684
Span‐level F1:        0.7694
Token‐level F1 per tag:
  B-PER: 0.9364
  I-PER: 0.9139
  B-LOC: 0.7940
  I-LOC: 0.7085
  B-ORG: 0.6098
  I-ORG: 0.7045
  O: 0.9921
Token‐level Macro F1: 0.8085
